# Merge native configurations (IOS)

In our previous example we merged two objects that already complied with the openconfig models. In this example, we are going to merge to native configurations.

Let's start by loading the driver as usual:

In [1]:
from ntc_rosetta import get_driver

ios = get_driver("ios", "openconfig")
ios_processor = ios()

Now we are going to load a file with the "running" configuration:

In [2]:
with open("data/ios/config.txt", "r") as f:
    running_config = f.read()

And now a different file with the "candidate" config:

In [3]:
with open("data/ios/new_config.txt", "r") as f:
    candidate_config = f.read()

Let's see the files side by side highlighting the differences:

In [4]:
!diff -y data/ios/config.txt data/ios/new_config.txt

hostname csr1000v-1						hostname csr1000v-1
!								!
ip name-server 8.8.8.8						ip name-server 8.8.8.8
!								!
ntp server 2610:20:6F96:96::6					ntp server 2610:20:6F96:96::6
ntp server 129.6.15.29						ntp server 129.6.15.29
!								!
ip ssh version 2						ip ssh version 2
ip ssh time-out 60						ip ssh time-out 60
!								!
username developer privilege 15 secret 9 $9$fhUXi6Xg438iAE$..	username developer privilege 15 secret 9 $9$fhUXi6Xg438iAE$..
username cisco privilege 15 secret 9 $9$COf3Q4xfzT.JxE$L3hvSk	username cisco privilege 15 secret 9 $9$COf3Q4xfzT.JxE$L3hvSk
username root privilege 15 secret 9 $9$FfUDIPBFcs03AE$MyLIWEu	username root privilege 15 secret 9 $9$FfUDIPBFcs03AE$MyLIWEu
username newuser80 password 0 Yu76_87AF				username newuser80 password 0 Yu76_87AF
!								!
interface FastEthernet1						interface FastEthernet1
   description This is Fa1					   description This is Fa1
   shutdown							   shutdown
   exit								   exit
!								!
interface FastEthernet

As you can see vlan 20 is gone, vlan 10 has been suspended and there is a new vlan 30.

Now let's parse those configurations as we did in our first tutorial:

In [5]:
parsed_candidate = ios_processor.parse(native={"dev_conf": candidate_config})
parsed_running = ios_processor.parse(native={"dev_conf": running_config})

Now that we have parsed both native configurations, doing a merge operation is identical as in our previous tutorial:

In [6]:
config = ios_processor.merge(
    candidate=parsed_candidate.raw_value(),
    running=parsed_running.raw_value()
)
print(config)

no vlan 20
vlan 10
   shutdown
   exit
!
vlan 30
   name staging
   no shutdown
   exit
!

